### Mount Drive and Import Drive Files

In [2]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parents[1]))
from Helper_Functions import common_utils
directory_to_extract_to = os.getcwd()

common_utils.load_data_from_one_drive(directory_to_extract_to, "recommendation_paths", "netflix_hybrid_movie")
common_utils.load_data_from_one_drive(directory_to_extract_to, "recommendation_paths", "netflix_hybrid_rating")

Extracting : 100%|██████████| 1/1 [00:34<00:00, 34.71s/it]


### Importing Libraries

In [6]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Collaborative Based Filtering

In [7]:
movie = pd.read_csv('movie.csv')
rating = pd.read_csv('rating.csv')

In [8]:
common_utils.get_dataset_info(movie)
common_utils.get_dataset_info(rating)

Head:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
Shape: (27278, 3)
Head:
   userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40
Shape: (20000263, 4)


In [9]:
movie_trim = movie[:1000]
common_utils.get_dataset_info(movie_trim)

Head:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
Shape: (1000, 3)


In [10]:
df = movie_trim.merge(rating, how="left", on="movieId")

In [11]:
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5,2009-01-02 01:13:41


In [12]:
common_utils.get_dataset_info(df)

Head:
   movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
1        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
2        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
3        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
4        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   

   userId  rating            timestamp  
0       3     4.0  1999-12-11 13:36:47  
1       6     5.0  1997-03-13 17:50:52  
2       8     4.0  1996-06-05 13:37:51  
3      10     4.0  1999-11-25 02:44:47  
4      11     4.5  2009-01-02 01:13:41  
Shape: (5399624, 6)


In [13]:
# we specify the parantheses so we don't conflict with movies that have years in their titles
df['year_movie'] = df.title.str.extract('(\(\d\d\d\d\))', expand=False)
#Removing the parentheses
df['year_movie'] = df.year_movie.str.extract('(\d\d\d\d)', expand=False)

#Removing the years from the 'title' column
df['title'] = df.title.str.replace('(\(\d\d\d\d\))', '',regex=True)
# To get rid of all the ending white space characters that might have appeared 
df['title'] = df['title'].apply(lambda x: x.strip())

a = pd.DataFrame(df["title"].value_counts())
rare_movies = a[a["title"] <= 1000].index
common_movies = df[~df["title"].isin(rare_movies)]
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

In [14]:
common_utils.get_dataset_info(a)

Head:
                           title
Pulp Fiction               67310
Forrest Gump               66172
Shawshank Redemption, The  63366
Silence of the Lambs, The  63299
Jurassic Park              59715
Shape: (998, 1)


In [15]:
rare_movies

Index(['Dunston Checks In', 'Safe', 'Big Green, The', 'Mark of Zorro, The',
       'Anne Frank Remembered', 'Foxfire', 'Party Girl',
       'Foreign Correspondent', 'Before the Rain (Pred dozhdot)',
       'Meet John Doe',
       ...
       'Gospa', 'Happy Weekend', 'Desert Winds', 'Two Crimes (Dos crímenes)',
       'Honey Moon (Honigmond)', 'Boy Called Hate, A', 'Roula',
       'Moonlight Murder', 'Girl in the Cadillac', 'Criminals'],
      dtype='object', length=426)

In [16]:
common_utils.get_dataset_info(common_movies)

Head:
   movieId      title                                       genres  userId  \
0        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy       3   
1        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy       6   
2        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy       8   
3        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy      10   
4        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy      11   

   rating            timestamp year_movie  
0     4.0  1999-12-11 13:36:47       1995  
1     5.0  1997-03-13 17:50:52       1995  
2     4.0  1996-06-05 13:37:51       1995  
3     4.0  1999-11-25 02:44:47       1995  
4     4.5  2009-01-02 01:13:41       1995  
Shape: (5263313, 7)


In [17]:
user_movie_df.head()

title,2 Days in the Valley,2001: A Space Odyssey,"39 Steps, The",Ace Ventura: Pet Detective,Ace Ventura: When Nature Calls,Addams Family Values,"Adventures of Pinocchio, The","Adventures of Priscilla, Queen of the Desert, The","Adventures of Robin Hood, The","Affair to Remember, An","African Queen, The","Age of Innocence, The",Airheads,Aladdin,All About Eve,All Dogs Go to Heaven 2,"Alphaville (Alphaville, une étrange aventure de Lemmy Caution)",Amateur,"American President, The","American in Paris, An",Angels and Insects,Angus,Another Stakeout,Antonia's Line (Antonia),"Apartment, The",Apollo 13,"Apple Dumpling Gang, The","Aristocats, The",Around the World in 80 Days,"Arrival, The",Assassins,Babe,"Babysitter, The",Backbeat,Bad Boys,Bad Girls,Balto,Barb Wire,Barbarella,Barcelona,"Basketball Diaries, The",Basquiat,Batman,Batman Forever,Beautiful Girls,Beauty and the Beast,Beauty of the Day (Belle de jour),Bed of Roses,Before Sunrise,Before and After,"Beverly Hillbillies, The",Beverly Hills Cop III,Beyond Rangoon,Big Night,Billy Madison,Bio-Dome,"Birdcage, The",Bitter Moon,Black Beauty,Black Sheep,Blade Runner,Blink,Bloodsport 2 (a.k.a. Bloodsport II: The Next Kumite),Blown Away,Blue in the Face,Body Snatchers,Boomerang,Bottle Rocket,Bound,Boxing Helena,Boys on the Side,"Brady Bunch Movie, The",Braveheart,Breakfast at Tiffany's,"Bridges of Madison County, The",Bringing Up Baby,Broken Arrow,"Bronx Tale, A","Brothers McMullen, The",Bulletproof,Bullets Over Broadway,Burnt by the Sun (Utomlyonnye solntsem),Cabin Boy,"Cable Guy, The",Canadian Bacon,Candyman: Farewell to the Flesh,Carlito's Way,Carrington,Casablanca,Casino,Casper,Cat on a Hot Tin Roof,"Celluloid Closet, The",Celtic Pride,Cemetery Man (Dellamorte Dellamore),Chain Reaction,"Chamber, The",Charade,Chungking Express (Chung Hing sam lam),Circle of Friends,Citizen Kane,City Hall,City Slickers II: The Legend of Curly's Gold,"City of Lost Children, The (Cité des enfants perdus, La)",Clean Slate,Clear and Present Danger,Clerks,"Client, The",Cliffhanger,Clockers,Clueless,Cobb,Cold Comfort Farm,Color of Night,Coneheads,Congo,Cops and Robbersons,Copycat,"Corrina, Corrina",Courage Under Fire,"Cowboy Way, The","Craft, The",Crimson Tide,Crooklyn,"Crossing Guard, The","Crow, The","Crow: City of Angels, The",Crumb,Cutthroat Island,D3: The Mighty Ducks,Dances with Wolves,Dangerous Minds,Dave,Daylight,Dazed and Confused,Dead Man,Dead Man Walking,Dead Presidents,Death and the Maiden,Demolition Man,Desperado,Devil in a Blue Dress,Diabolique,Die Hard: With a Vengeance,Disclosure,Dolores Claiborne,Don Juan DeMarco,Don't Be a Menace to South Central While Drinking Your Juice in the Hood,Down Periscope,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,Dracula: Dead and Loving It,Dragonheart,Drop Zone,Dumb & Dumber (Dumb and Dumber),East of Eden,Eat Drink Man Woman (Yin shi nan nu),Ed Wood,Eddie,Emma,"Englishman Who Went Up a Hill But Came Down a Mountain, The",Eraser,Escape from L.A.,Escape to Witch Mountain,Even Cowgirls Get the Blues,Executive Decision,Exit to Eden,Exotica,Extreme Measures,Eye for an Eye,Fair Game,"Family Thing, A","Fan, The",Farewell My Concubine (Ba wang bie ji),Fargo,Farinelli: il castrato,Fatal Instinct,Father of the Bride,Father of the Bride Part II,Fear,Fearless,Feeling Minnesota,"Firm, The",First Kid,First Knight,"First Wives Club, The",Fled,"Flintstones, The",Flipper,Flirting With Disaster,Fly Away Home,Forget Paris,Forrest Gump,Four Rooms,Four Weddings and a Funeral,Free Willy,Free Willy 2: The Adventure Home,French Kiss,French Twist (Gazon maudit),Friday,"Frighteners, The",From Dusk Till Dawn,"Fugitive, The",Funny Face,Gaslight,Geronimo: An American Legend,Get Shorty,"Getaway, The",Ghost,"Ghost and Mrs. Muir, The",Ghost in the Shell (Kôkaku kidôtai),Giant,Gigi,Girl 6,"Glimmer Man, The","Godfather, The",GoldenEye,Gone with the Wind,"Goofy Movie, A","Great White Hype, The",Grumpier Old Men,Hackers,Halloween: The Curse of Michael Myers (Halloween 6: Th

In [19]:
user_movie_df.shape

(137369, 572)

In [20]:
# choosing random a user
user_id = int(pd.Series(user_movie_df.index).sample(1).values)
print("Selected user: ", user_id)

Selected user:  55881


In [21]:
# Determining the movies watched by the recommended user
user_df = user_movie_df[user_movie_df.index == user_id]

movies_watched = user_df.columns[user_df.notna().any()].tolist()
print("No. of movies watched by user: ", len(movies_watched))

No. of movies watched by user:  37


In [22]:
user_df

title,2 Days in the Valley,2001: A Space Odyssey,"39 Steps, The",Ace Ventura: Pet Detective,Ace Ventura: When Nature Calls,Addams Family Values,"Adventures of Pinocchio, The","Adventures of Priscilla, Queen of the Desert, The","Adventures of Robin Hood, The","Affair to Remember, An","African Queen, The","Age of Innocence, The",Airheads,Aladdin,All About Eve,All Dogs Go to Heaven 2,"Alphaville (Alphaville, une étrange aventure de Lemmy Caution)",Amateur,"American President, The","American in Paris, An",Angels and Insects,Angus,Another Stakeout,Antonia's Line (Antonia),"Apartment, The",Apollo 13,"Apple Dumpling Gang, The","Aristocats, The",Around the World in 80 Days,"Arrival, The",Assassins,Babe,"Babysitter, The",Backbeat,Bad Boys,Bad Girls,Balto,Barb Wire,Barbarella,Barcelona,"Basketball Diaries, The",Basquiat,Batman,Batman Forever,Beautiful Girls,Beauty and the Beast,Beauty of the Day (Belle de jour),Bed of Roses,Before Sunrise,Before and After,"Beverly Hillbillies, The",Beverly Hills Cop III,Beyond Rangoon,Big Night,Billy Madison,Bio-Dome,"Birdcage, The",Bitter Moon,Black Beauty,Black Sheep,Blade Runner,Blink,Bloodsport 2 (a.k.a. Bloodsport II: The Next Kumite),Blown Away,Blue in the Face,Body Snatchers,Boomerang,Bottle Rocket,Bound,Boxing Helena,Boys on the Side,"Brady Bunch Movie, The",Braveheart,Breakfast at Tiffany's,"Bridges of Madison County, The",Bringing Up Baby,Broken Arrow,"Bronx Tale, A","Brothers McMullen, The",Bulletproof,Bullets Over Broadway,Burnt by the Sun (Utomlyonnye solntsem),Cabin Boy,"Cable Guy, The",Canadian Bacon,Candyman: Farewell to the Flesh,Carlito's Way,Carrington,Casablanca,Casino,Casper,Cat on a Hot Tin Roof,"Celluloid Closet, The",Celtic Pride,Cemetery Man (Dellamorte Dellamore),Chain Reaction,"Chamber, The",Charade,Chungking Express (Chung Hing sam lam),Circle of Friends,Citizen Kane,City Hall,City Slickers II: The Legend of Curly's Gold,"City of Lost Children, The (Cité des enfants perdus, La)",Clean Slate,Clear and Present Danger,Clerks,"Client, The",Cliffhanger,Clockers,Clueless,Cobb,Cold Comfort Farm,Color of Night,Coneheads,Congo,Cops and Robbersons,Copycat,"Corrina, Corrina",Courage Under Fire,"Cowboy Way, The","Craft, The",Crimson Tide,Crooklyn,"Crossing Guard, The","Crow, The","Crow: City of Angels, The",Crumb,Cutthroat Island,D3: The Mighty Ducks,Dances with Wolves,Dangerous Minds,Dave,Daylight,Dazed and Confused,Dead Man,Dead Man Walking,Dead Presidents,Death and the Maiden,Demolition Man,Desperado,Devil in a Blue Dress,Diabolique,Die Hard: With a Vengeance,Disclosure,Dolores Claiborne,Don Juan DeMarco,Don't Be a Menace to South Central While Drinking Your Juice in the Hood,Down Periscope,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,Dracula: Dead and Loving It,Dragonheart,Drop Zone,Dumb & Dumber (Dumb and Dumber),East of Eden,Eat Drink Man Woman (Yin shi nan nu),Ed Wood,Eddie,Emma,"Englishman Who Went Up a Hill But Came Down a Mountain, The",Eraser,Escape from L.A.,Escape to Witch Mountain,Even Cowgirls Get the Blues,Executive Decision,Exit to Eden,Exotica,Extreme Measures,Eye for an Eye,Fair Game,"Family Thing, A","Fan, The",Farewell My Concubine (Ba wang bie ji),Fargo,Farinelli: il castrato,Fatal Instinct,Father of the Bride,Father of the Bride Part II,Fear,Fearless,Feeling Minnesota,"Firm, The",First Kid,First Knight,"First Wives Club, The",Fled,"Flintstones, The",Flipper,Flirting With Disaster,Fly Away Home,Forget Paris,Forrest Gump,Four Rooms,Four Weddings and a Funeral,Free Willy,Free Willy 2: The Adventure Home,French Kiss,French Twist (Gazon maudit),Friday,"Frighteners, The",From Dusk Till Dawn,"Fugitive, The",Funny Face,Gaslight,Geronimo: An American Legend,Get Shorty,"Getaway, The",Ghost,"Ghost and Mrs. Muir, The",Ghost in the Shell (Kôkaku kidôtai),Giant,Gigi,Girl 6,"Glimmer Man, The","Godfather, The",GoldenEye,Gone with the Wind,"Goofy Movie, A","Great White Hype, The",Grumpier Old Men,Hackers,Halloween: The Curse of Michael Myers (Halloween 6: Th

In [23]:
# Movies watched by user
movies_watched

['Ace Ventura: Pet Detective',
 'Airheads',
 'Aladdin',
 'Babe',
 'Batman',
 'Batman Forever',
 'Billy Madison',
 'Crimson Tide',
 'Dances with Wolves',
 'Dangerous Minds',
 'Dumb & Dumber (Dumb and Dumber)',
 'Forrest Gump',
 'Fugitive, The',
 'Ghost',
 'Home Alone',
 'Hot Shots! Part Deux',
 'Junior',
 'Jurassic Park',
 'Lion King, The',
 'Mask, The',
 'Miracle on 34th Street',
 'Mrs. Doubtfire',
 'Naked Gun 33 1/3: The Final Insult',
 'Net, The',
 'Operation Dumbo Drop',
 'Robin Hood: Men in Tights',
 'RoboCop 3',
 "Schindler's List",
 'Silence of the Lambs, The',
 'So I Married an Axe Murderer',
 'Speed',
 'Star Trek: Generations',
 'Super Mario Bros.',
 'Terminator 2: Judgment Day',
 'Three Musketeers, The',
 'Tommy Boy',
 'True Lies']

In [25]:
# Keeping only movies watched by given user
movies_watched_df = user_movie_df[movies_watched]

movies_watched_df.shape

(137369, 37)

In [26]:
movies_watched_df.head()

title,Ace Ventura: Pet Detective,Airheads,Aladdin,Babe,Batman,Batman Forever,Billy Madison,Crimson Tide,Dances with Wolves,Dangerous Minds,Dumb & Dumber (Dumb and Dumber),Forrest Gump,"Fugitive, The",Ghost,Home Alone,Hot Shots! Part Deux,Junior,Jurassic Park,"Lion King, The","Mask, The",Miracle on 34th Street,Mrs. Doubtfire,Naked Gun 33 1/3: The Final Insult,"Net, The",Operation Dumbo Drop,Robin Hood: Men in Tights,RoboCop 3,Schindler's List,"Silence of the Lambs, The",So I Married an Axe Murderer,Speed,Star Trek: Generations,Super Mario Bros.,Terminator 2: Judgment Day,"Three Musketeers, The",Tommy Boy,True Lies
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,4.0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,4.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,4.0,3.0,NaN,4.0,NaN,NaN,3.0
5,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,4.0,5.0,3.0,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,5.0,NaN,NaN,5.0,NaN,NaN,5.0


In [27]:
# each user watched how many of these movies
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count.head()

userId
1     3
2     2
3     5
4    11
5    13
dtype: int64

In [28]:
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]
m_count = movies_watched_df.shape[1]

# choosen a ratio of 0.60. Users who watched at least 60 percent of these movies 
users_same_movies=user_movie_count[user_movie_count["movie_count"]/m_count > 0.6].sort_values("movie_count", ascending=False)
users_same_movies.nunique()

userId         8268
movie_count      15
dtype: int64

In [29]:
print("%d users have watched 60 percent same movies wrt choosen user" %(users_same_movies.shape[0]))

8268 users have watched 60 percent same movies wrt choosen user


In [30]:
# We bring together users who watch the same movies with the user. (not overlook the above ratio)
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies.index)],
                      user_df[movies_watched]])

In [31]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df.head()

,user_id_1,user_id_2,corr
0,78006,92254,-1.0
1,114979,43221,-1.0
2,71202,1606,-1.0
3,131308,59060,-1.0
4,53166,43984,-1.0


In [32]:
corr_df.shape[0]

1867000

In [33]:
top_users = corr_df[(corr_df["user_id_1"] == user_id) & (corr_df["corr"] >= 0.60)][
    ["user_id_2", "corr"]].reset_index(drop=True)

top_users = top_users.sort_values(by='corr', ascending=False)

# Users who have more than 0.60 corr with the user
top_users.rename(columns={"user_id_2": "userId"}, inplace=True) 
top_users.T

,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0
userId,86100.000000,32127.000000,104080.000000,82232.000000,14331.000000,15800.000000,4857.000000,5263.00000,62302.00000,91809.000000,10537.000000,75760.000000,126687.000000,83309.00000,85564.000000
corr,0.908739,0.869926,0.792825,0.773117,0.741561,0.724743,0.688838,0.65625,0.64715,0.640445,0.623479,0.620554,0.611111,0.60943,0.602111


Do join of movies and rating and use here

In [73]:
# By doing this we are only using ratings of the selected 1000 movies
rating_new = movie_trim.merge(rating, how="left", on="movieId")

In [74]:
common_utils.get_dataset_info(rating_new)

Head:
   movieId          movie_data   userId  rating            timestamp
0   102895          drama 2012  13494.0     3.5  2013-06-01 16:00:33
1   110123    documentary 2010  42640.0     3.5  2014-03-28 07:46:35
2   110123    documentary 2010  61168.0     1.5  2014-04-08 12:00:07
3   121101          drama 1949      NaN     NaN                  NaN
4     6088  comedy sci-fi 1982   4222.0     1.0  2003-01-21 15:02:03
Shape: (868522, 5)


In [75]:
# Calculation of weighted rating
top_users_ratings = top_users.merge(rating_new[["userId", "movieId", "rating"]], how='inner')
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
common_utils.get_dataset_info(top_users_ratings)

Head:
   userId      corr  movieId  rating  weighted_rating
0   86100  0.908739     4720     3.5         3.180586
1   86100  0.908739     5247     3.5         3.180586
2   86100  0.908739     2959     3.5         3.180586
3   32127  0.869926      225     3.0         2.609779
4   32127  0.869926      457     5.0         4.349631
Shape: (78, 5)


In [76]:
# Calculating the weighted average recommendation score and keeping the top ten films
# Unique rating for movieId 
temp = top_users_ratings.groupby('movieId').sum()[['corr', 'weighted_rating']]
temp.columns = ['sum_corr', 'sum_weighted_rating']
temp.head()

,sum_corr,sum_weighted_rating
movieId,,
50,4.651273,19.710193
225,3.014936,11.962936
247,0.741561,3.337025
260,2.573317,9.718180
457,6.139764,26.616368


In [77]:
# Calculating weighted average recommendation score and keeping top five films based on scores
recommendation_df = pd.DataFrame()
recommendation_df['weighted_average_recommendation_score'] = temp['sum_weighted_rating'] / temp['sum_corr']
recommendation_df['movieId'] = temp.index
recommendation_df = recommendation_df.sort_values(by='weighted_average_recommendation_score', ascending=False)
recommendation_df.head()

,weighted_average_recommendation_score,movieId
movieId,,
509,5.0,509
57502,5.0,57502
1178,5.0,1178
3011,5.0,3011
247,4.5,247


Movie recommendations that the user may like

In [78]:
movie.loc[movie['movieId'].isin(recommendation_df.head(10)['movieId'])]

,movieId,title,genres
49,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
244,247,Heavenly Creatures (1994),Crime|Drama
453,457,"Fugitive, The (1993)",Thriller
505,509,"Piano, The (1993)",Drama|Romance
1155,1178,Paths of Glory (1957),Drama|War
1913,1997,"Exorcist, The (1973)",Horror|Mystery
1983,2067,Doctor Zhivago (1965),Drama|Romance|War
2925,3011,"They Shoot Horses, Don't They? (1969)",Drama
9556,27904,"Scanner Darkly, A (2006)",Animation|Drama|Mystery|Sci-Fi|Thriller
12393,57502,Cat Soup (Nekojiru-so) (2001),Adventure|Animation|Drama|Horror


### Content based filtering (Edited)

In [39]:
movie_df = pd.read_csv('movie.csv')
movie_df.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [40]:
# Cleaning movie titles(removing year of movie and adding that to a new column)

movie_df['year_movie'] = movie_df.title.str.extract('(\(\d\d\d\d\))', expand=False)
movie_df['year_movie'] = movie_df.year_movie.str.extract('(\d\d\d\d)', expand=False)

#Removing the years from the 'title' column
movie_df['title'] = movie_df.title.str.replace('(\(\d\d\d\d\))', '',regex=True)
# To get rid of all the ending white space characters that might have appeared 
movie_df['title'] = movie_df['title'].apply(lambda x: x.strip())

In [41]:
movie_df.head()

,movieId,title,genres,year_movie
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [42]:
# Replacing "|"" with " " in genres
movie_df["genres"] = movie_df["genres"].apply(lambda x: x.replace('|', ' '))

In [43]:
movie_df.head()

,movieId,title,genres,year_movie
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995
1,2,Jumanji,Adventure Children Fantasy,1995
2,3,Grumpier Old Men,Comedy Romance,1995
3,4,Waiting to Exhale,Comedy Drama Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [44]:
# Checking for any nan values in year of movies
movie_df.year_movie.unique()

array(['1995', '1994', '1996', '1976', '1992', '1988', '1967', '1993',
       '1964', '1977', '1965', '1982', '1985', '1990', '1991', '1989',
       '1937', '1940', '1969', '1981', '1973', '1970', '1960', '1955',
       '1959', '1968', '1980', '1975', '1986', '1948', '1943', '1950',
       '1946', '1987', '1997', '1974', '1956', '1958', '1949', '1972',
       '1998', '1933', '1952', '1951', '1957', '1961', '1954', '1934',
       '1944', '1963', '1942', '1941', '1953', '1939', '1947', '1945',
       '1938', '1935', '1936', '1926', '1932', '1979', '1971', '1978',
       '1966', '1962', '1983', '1984', '1931', '1922', '1999', '1927',
       '1929', '1930', '1928', '1925', '1914', '2000', '1919', '1923',
       '1920', '1918', '1921', '2001', '1924', '2002', '2003', '1915',
       '2004', '1916', '1917', '2005', '2006', '1902', nan, '1903',
       '2007', '2008', '2009', '1912', '2010', '1913', '2011', '1898',
       '1899', '1894', '2012', '1909', '1910', '1901', '1893', '2013',
       '1

In [45]:
movie_df['year_movie'].isna().sum()

22

In [46]:
movie_df.shape

(27278, 4)

In [47]:
common_utils.get_count_and_percentage_missing_values(movie_df)

            count  % null
movieId         0     0.0
title           0     0.0
genres          0     0.0
year_movie     22     0.1


In [48]:
movie_df = movie_df.dropna()

In [49]:
movie_df.shape

(27256, 4)

In [50]:
common_utils.get_count_and_percentage_missing_values(movie_df)

            count  % null
movieId         0     0.0
title           0     0.0
genres          0     0.0
year_movie      0     0.0


In [51]:
movie_df.head()

,movieId,title,genres,year_movie
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995
1,2,Jumanji,Adventure Children Fantasy,1995
2,3,Grumpier Old Men,Comedy Romance,1995
3,4,Waiting to Exhale,Comedy Drama Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [52]:
# Combining genre and year of movie into one as movie data
movie_df['movie_data'] = movie_df['genres'] + ' ' + movie_df['year_movie']

In [53]:
movie_df.head()

,movieId,title,genres,year_movie,movie_data
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995,Adventure Animation Children Comedy Fantasy 1995
1,2,Jumanji,Adventure Children Fantasy,1995,Adventure Children Fantasy 1995
2,3,Grumpier Old Men,Comedy Romance,1995,Comedy Romance 1995
3,4,Waiting to Exhale,Comedy Drama Romance,1995,Comedy Drama Romance 1995
4,5,Father of the Bride Part II,Comedy,1995,Comedy 1995


In [54]:
movie_df['movie_data'] = movie_df['movie_data'].str.lower()

In [55]:
movie_df.head()

,movieId,title,genres,year_movie,movie_data
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995,adventure animation children comedy fantasy 1995
1,2,Jumanji,Adventure Children Fantasy,1995,adventure children fantasy 1995
2,3,Grumpier Old Men,Comedy Romance,1995,comedy romance 1995
3,4,Waiting to Exhale,Comedy Drama Romance,1995,comedy drama romance 1995
4,5,Father of the Bride Part II,Comedy,1995,comedy 1995


In [56]:
movie_filtered_df = movie_df.filter(['movieId','movie_data'], axis=1)

In [57]:
movie_filtered_df.head()

,movieId,movie_data
0,1,adventure animation children comedy fantasy 1995
1,2,adventure children fantasy 1995
2,3,comedy romance 1995
3,4,comedy drama romance 1995
4,5,comedy 1995


In [58]:
movie_filtered_df.set_index('movieId', inplace = True)

In [59]:
movie_filtered_df.head()

,movie_data
movieId,
1,adventure animation children comedy fantasy 1995
2,adventure children fantasy 1995
3,comedy romance 1995
4,comedy drama romance 1995
5,comedy 1995


In [60]:
movie_trim = movie_filtered_df.sample(n = 1000)

In [61]:
movie_trim.shape

(1000, 1)

In [62]:
# instantiating and generating the count matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count = CountVectorizer()
count_matrix = count.fit_transform(movie_trim['movie_data'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [63]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(movie_trim.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(movie_trim.index)[i])
        
    return recommended_movies

In [64]:
movie_random = movie_trim.sample()

In [65]:
movie_random

,movie_data
movieId,
51800,drama war 2000


In [66]:
movie_random_ind = movie_random.index.values.astype(int)[0]

In [67]:
movie_random_ind

51800

In [68]:
recommended_movies_ind = recommendations(movie_random_ind)

In [69]:
recommended_movies_ind

[4729, 5221, 3967, 4019, 72267, 108313, 3915, 78068, 78445, 27015]

In [70]:
movie_chosen = movie_df[movie_df['movieId'] == movie_random_ind]['title'].values.astype(str)[0]

print("Choosen movie: ", movie_chosen)

Choosen movie:  Kippur


In [71]:
recommend_movies = []
for movie_ind in recommended_movies_ind:
  rec_movie = movie_df[movie_df['movieId'] == movie_ind]['title'].values.astype(str)[0]
  recommend_movies.append(rec_movie)

In [72]:
recommend_movies

['Aberdeen',
 "Harrison's Flowers",
 'Billy Elliot',
 'Finding Forrester',
 'Price of Glory',
 'Dirty Pictures',
 'Girlfight',
 'Aerial Gunner',
 'Woman In Berlin, A (Anonyma - Eine Frau in Berlin)',
 'Bruno (Dress Code, The)']